Accelerating portable HPC Applications with Standard C++
===

# Lab 1: select

If [Lab 1: DAXPY](../lab1_daxpy/daxpy.ipynb) was quick to complete for you, Lab 1 proposes a slightly more advanced example which requires the decomposition of a problem into multiple algorithm calls. You will use different approaches, sequential and parallel, to write a function `select` which selects some elements of an input vector `v` according to a general, user-provided criterion and copies the selected element consecutively into a new vector `w`.

This problem is easy to solve sequentially but faces an issue in a concurrent run: the index of write operations into `w` depends on operations performed by other threads.

## Initial condition

For all the exercises, the vector `v` is filled with pseudo-random numbers that are seeded with a constant value and are therefore identical from one execution to another.



## Exercise 1: serial implementation with `std::copy_if`

The goal of this first exercise is to write a version of `select` which calls the algorithm `copy_if`. It is simple an elegant, but not parallelizable.

A template for the solution is provided in [exercise1.cpp]. The `TODO` indicates the part of the template that must be completed.
A first version of `select` is provided, which looks as follows:

```c++
template<class UnaryPredicate>
std::vector<int> select(const std::vector<int>& v, UnaryPredicate pred)
{
    // TODO Instead of the line below, create a vector w and use a "copy_if" algorithm
    // call to copy all elements from v to w that are selected by the unary predicate.
    auto w = v;
    return w;
}
```

[exercise1.cpp]: ./exercise1.cpp

The example compiles and runs as provided, but it produces incorrect results due to the erroneous `select` implementation.
Replace the erroneous line by an appropriate call to the `copy_if` algorithm.
Hint: You can't allocate the right number of elements for `w` in advance, because you don't know how many elements the `copy_if` algorithm is going to copy. Instead, use `std::back_inserter` to create an iterator which inserts elements at the back of `w` and resizes the vector appropriately as the algorithm progresses.
Once you fix the code, the following block should compile and run correctly:



In [ ]:
!g++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise1.cpp
!./select 30

In [ ]:
!clang++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise1.cpp
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise1.cpp
!./select 30

### Solutions Exercise 1

The solutions for each example are available in the [`solutions/`] sub-directory.

[`solutions/`]: ./solutions

The solution for this first exercise is available at [`solutions/exercise1.cpp`].

[`solutions/exercise1.cpp`]: ./solutions/exercise1.cpp

The following compiles and runs the solutions for Exercise 1 using different compilers and C++ standard versions.

In [ ]:
!g++ -std=c++20 -Ofast -march=native -DNDEBUG -o select solutions/exercise1.cpp
!./select 30

In [ ]:
!clang++ -std=c++20 -Ofast -march=native -DNDEBUG -o select solutions/exercise1.cpp
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -o select solutions/exercise1.cpp
!./select 30

## Exercise 2: parallel implementation with `std::transform`, `std::inclusive_scan`, and `std::for_each`

The code of exercise 1 cannot run in parallel, because the back inserter adds elements to `w` sequentially.
Open [exercise2.cpp], look out for the `TODO` comment and correct the code by proceeding in three steps:
1. Use `transform` to create a vector `v_sel` which has the same length as `v` and is filled with 0/1 values, depending on the result of the unary predicate applied to the corresponding element of `v`.
2. Use `inclusive_scan` to compute the cumulative sum of `v_sel` and store the result into the vector `index`, which will provide indices of the selected elements of `v` into `w`. *Attention: with `inclusive_scan`, the indices are off by one. We wouldn't have this off-by-one error with `exclusive_scan`, but `inlusive_scan` is quite convenient here: its last element indicates the total number of selected elements, and thus, the size of elements to allocate for `w`*.
3. Use a `for_each` statement to copy values from `v` to `w`, depending on the outcome of the unary predicate.

Once the code is completed, the following blocks should complete properly and produce the same output as in exercise 0.

[exercise2.cpp]: ./exercise2.cpp



In [ ]:
!g++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise2.cpp -ltbb
!./select 30

In [ ]:
!clang++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise2.cpp
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=multicore -o select exercise2.cpp
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=gpu -o select exercise2.cpp
!./select 30

### Solutions Exercise 2

The solutions for each example are available at [`solutions/exercise2.cpp`].

[`solutions/exercise2.cpp`]: ./solutions/exercise2.cpp

The following compiles and runs the solutions for Exercise 1 using different compilers and C++ standard versions.

In [ ]:
!g++ -std=c++20 -Ofast -march=native -DNDEBUG -o select solutions/exercise2.cpp -ltbb
!./select 30

In [ ]:
!clang++ -std=c++20 -Ofast -march=native -DNDEBUGG -o select solutions/exercise2.cpp -ltbb
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=multicore -o select solutions/exercise2.cpp
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=gpu -o select solutions/exercise2.cpp
!./select 30

## Exercise 3

In exercise 2, we decomposed the selection process in three algorithm calls to clearly illustrate the different steps involved in the parallelizable approach. But C++ offers the algorithm `transform_inclusive_scan` which combines the two first steps and avoids the need for allocating the vector `v_sel`.
Open [exercise3.cpp], look out for the `TODO` comment and implement `select` through a parallelizable approach as before, but in two steps, using `transform_inclusive_scan`.

In this exercise, the API of `select` has been improved for performance as follows:

```c++
template<class UnaryPredicate>
void select(const std::vector<int>& v, UnaryPredicate pred,
            std::vector<size_t>& index, std::vector<int>& w)
{
    // TODO: write a parallelizable version of select, just as for exercise 1.
    // But this time:
    // - accept index and w as inputs to allow re-using the buffers
    // - use transform_inclusive_scan to reduce the number of steps from 3 to 2.   
}
```

This enables the user of the API to pass the algorithm pre-allocated storage buffers for the `index` and the result `w`, enabling applications to re-use these buffers across `select` calls.

This exercise also includes a benchmarking function `bench` that computes the memory bandwidth achieved by the different implementations.

Once the code is completed, the following blocks should complete properly and produce the same output as in exercise 1 and exercise 2.

[exercise3.cpp]: ./exercise3.cpp



In [ ]:
!g++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise3.cpp -ltbb
!./select 30

In [ ]:
!clang++ -std=c++20 -Ofast -march=native -DNDEBUG -o select exercise3.cpp -ltbb
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=multicore -o select exercise3.cpp
!./select 30

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=gpu -o select exercise3.cpp
!./select 30

### Solutions Exercise 3

The solutions for each example are available at [`solutions/exercise3.cpp`].

[`solutions/exercise3.cpp`]: ./solutions/exercise3.cpp

The following compiles and runs the solutions for Exercise 3 using different compilers and C++ standard versions.

In [ ]:
!g++ -std=c++20 -Ofast -march=native -DNDEBUG -o select solutions/exercise3.cpp -ltbb
!./select 10000000

In [ ]:
!clang++ -std=c++20 -Ofast -march=native -DNDEBUG -o select solutions/exercise3.cpp -ltbb
!./select 10000000

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=multicore -o select solutions/exercise3.cpp
!./select 10000000

In [ ]:
!nvc++ -std=c++20 -Ofast -march=native -DNDEBUG -stdpar=gpu -o select solutions/exercise3.cpp
!./select 10000000